## Read zarr data from s3

Analysis Of Record for Calbration dataset

 - [Description](https://registry.opendata.aws/noaa-nws-aorc/)
 - [Browse Data](https://noaa-nws-aorc-v1-1-1km.s3.amazonaws.com/index.html)

In [1]:
import xarray as xr

In [2]:
def s3_key_aorc(year:int, bucket_name:str="noaa-nws-aorc-v1-1-1km")->str:
    """Return s3 key of zarr data for a given year."""
    return  f"s3://{bucket_name}/{year}.zarr"

### Read in a single year of data

In [3]:
key = s3_key_aorc(1979)
print(key)

s3://noaa-nws-aorc-v1-1-1km/1979.zarr


In [4]:
ds = xr.open_dataset(key, engine="zarr")
ds['TMP_2maboveground']

<xarray.DataArray 'TMP_2maboveground' (time: 8016, latitude: 4201,
                                       longitude: 8401)> Size: 2TB
[282905489616 values with dtype=float64]
Coordinates:
  * latitude   (latitude) float64 34kB 20.0 20.01 20.02 ... 54.98 54.99 55.0
  * longitude  (longitude) float64 67kB -130.0 -130.0 -130.0 ... -60.01 -60.0
  * time       (time) datetime64[ns] 64kB 1979-02-01 ... 1979-12-31T23:00:00
Attributes:
    AORC_Contact:  aorc.info@noaa.gov
    aorc_version:  v1.1
    crs:           EPSG:4326
    level:         2 m above ground
    long_name:     Temperature
    short_name:    TMP_2maboveground
    units:         K

### Read in multiple years of data

In [5]:
# Define start / end year 
start_year, end_year = 1979, 1982

# Create list of years
years = range(start_year, end_year)

# Create list of s3 keys
keys = [s3_key_aorc(y) for y in years]

print(keys)


['s3://noaa-nws-aorc-v1-1-1km/1979.zarr', 's3://noaa-nws-aorc-v1-1-1km/1980.zarr', 's3://noaa-nws-aorc-v1-1-1km/1981.zarr']


In [6]:
# Read in all years of data
ds = xr.open_mfdataset(keys, engine="zarr", consolidated=True, parallel=True)
ds['TMP_2maboveground']

<xarray.DataArray 'TMP_2maboveground' (time: 25560, latitude: 4201,
                                       longitude: 8401)> Size: 7TB
dask.array<concatenate, shape=(25560, 4201, 8401), dtype=float64, chunksize=(144, 128, 256), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 34kB 20.0 20.01 20.02 ... 54.98 54.99 55.0
  * longitude  (longitude) float64 67kB -130.0 -130.0 -130.0 ... -60.01 -60.0
  * time       (time) datetime64[ns] 204kB 1979-02-01 ... 1981-12-31T23:00:00
Attributes:
    AORC_Contact:  aorc.info@noaa.gov
    aorc_version:  v1.1
    crs:           EPSG:4326
    level:         2 m above ground
    long_name:     Temperature
    short_name:    TMP_2maboveground
    units:         K